In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [159]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [160]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0-rc0


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [163]:
# Parameters for the model and dataset.
TRAINING_SIZE = 1000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int / int' (e.g., '345/678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789/ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    if b == 0: continue
    q = '{}/{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    # truncate decimal place by using int()
    ans = str(int(a / b))
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12/345  ' becomes '  543/21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 1000


In [164]:
questions[0]

'388/4  '

In [165]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS)


Vectorization...


In [166]:
len(x[0])

7

In [167]:
len(questions[0])

7

In [168]:
questions[0]

'388/4  '

In [169]:
x[0]

array([[False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [170]:
y[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [171]:
expected[0]

'97 '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [173]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(900, 7, 12)
(900, 3, 12)
Validation Data:
(100, 7, 12)
(100, 3, 12)


### Step 3: Create Model

In [174]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [175]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_9 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 3, 128)            0         
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 3, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 3, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [176]:
model.predict(np.array([x_val[0]]))

array([[[0.05959769, 0.11379386, 0.07196995, 0.06314506, 0.09937552,
         0.10211429, 0.11341094, 0.06833564, 0.05278398, 0.07822824,
         0.09549123, 0.08175357],
        [0.04555275, 0.09948279, 0.06545865, 0.05955165, 0.12404118,
         0.11866523, 0.1932387 , 0.05100052, 0.04127151, 0.08646683,
         0.05420032, 0.06106989],
        [0.04522731, 0.09675682, 0.05498974, 0.04581303, 0.13071683,
         0.14020629, 0.15408897, 0.02441037, 0.04543772, 0.1394013 ,
         0.04000411, 0.08294752]]], dtype=float32)

In [177]:
model.predict_classes(np.array([x_val[0]]))

array([[1, 6, 6]])

### Step 4: Train

In [178]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 1s 1ms/step - loss: 1.9088 - acc: 0.4059 - val_loss: 1.1317 - val_acc: 0.6733
Q 53/103  T 0   ☒    
Q 56/67   T 0   ☒    
Q 6/92    T 0   ☑ 0  
Q 57/4    T 14  ☒    
Q 803/82  T 9   ☒ 0  
Q 6/887   T 0   ☒    
Q 674/33  T 20  ☒    
Q 2/9     T 0   ☑ 0  
Q 1/27    T 0   ☒    
Q 699/283 T 2   ☒ 1  

--------------------------------------------------
Iteration 2
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 138us/step - loss: 1.1611 - acc: 0.7163 - val_loss: 0.9798 - val_acc: 0.7667
Q 1/53    T 0   ☑ 0  
Q 64/965  T 0   ☑ 0  
Q 8/358   T 0   ☑ 0  
Q 310/46  T 6   ☒ 0  
Q 0/850   T 0   ☑ 0  
Q 9/512   T 0   ☑ 0  
Q 9/512   T 0   ☑ 0  
Q 1/1     T 1   ☒ 0  
Q 7/496   T 0   ☑ 0  
Q 27/838  T 0   ☑ 0  

--------------------------------------------------
Iteration 3
Train on 900

900/900 [==============================] - 0s 185us/step - loss: 0.7982 - acc: 0.7737 - val_loss: 0.7849 - val_acc: 0.7867
Q 686/8   T 85  ☒ 1  
Q 1/577   T 0   ☑ 0  
Q 7/496   T 0   ☑ 0  
Q 93/16   T 5   ☒ 1  
Q 27/9    T 3   ☒ 0  
Q 6/887   T 0   ☑ 0  
Q 0/607   T 0   ☑ 0  
Q 310/46  T 6   ☒ 0  
Q 176/711 T 0   ☑ 0  
Q 1/2     T 0   ☑ 0  

--------------------------------------------------
Iteration 9
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 188us/step - loss: 0.7762 - acc: 0.7711 - val_loss: 0.7580 - val_acc: 0.7867
Q 310/46  T 6   ☒ 0  
Q 59/3    T 19  ☒ 1  
Q 478/8   T 59  ☒ 14 
Q 12/7    T 1   ☒ 0  
Q 7/496   T 0   ☑ 0  
Q 71/983  T 0   ☑ 0  
Q 308/337 T 0   ☑ 0  
Q 47/1    T 47  ☒ 0  
Q 85/21   T 4   ☒ 1  
Q 8/770   T 0   ☑ 0  

--------------------------------------------------
Iteration 10
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 221us/step - loss: 0.7398 -

Q 628/873 T 0   ☒ 1  
Q 8/985   T 0   ☑ 0  
Q 6/887   T 0   ☑ 0  
Q 445/49  T 9   ☒ 1  
Q 64/965  T 0   ☑ 0  
Q 1/53    T 0   ☑ 0  
Q 27/9    T 3   ☒ 0  
Q 47/1    T 47  ☒ 1  
Q 2/9     T 0   ☑ 0  
Q 444/585 T 0   ☒ 1  

--------------------------------------------------
Iteration 16
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 182us/step - loss: 0.6300 - acc: 0.7985 - val_loss: 0.6337 - val_acc: 0.8067
Q 8/985   T 0   ☑ 0  
Q 688/63  T 10  ☒ 1  
Q 558/19  T 29  ☒ 10 
Q 176/711 T 0   ☑ 0  
Q 5/55    T 0   ☑ 0  
Q 12/7    T 1   ☑ 1  
Q 4/77    T 0   ☑ 0  
Q 954/2   T 477 ☒ 133
Q 954/2   T 477 ☒ 133
Q 327/919 T 0   ☑ 0  

--------------------------------------------------
Iteration 17
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 180us/step - loss: 0.6261 - acc: 0.7974 - val_loss: 0.6666 - val_acc: 0.7967
Q 91/99   T 0   ☑ 0  
Q 77/8    T 9   ☒ 1  
Q 1/2     T 0   ☑ 0  
Q 802/

In [179]:
x_val[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [180]:
model.predict(np.array([x_val[0]]))

array([[[3.3120678e-03, 3.0454202e-04, 9.7121531e-01, 5.6328918e-03,
         2.9835203e-03, 1.7616936e-03, 3.6319050e-03, 1.3943801e-03,
         2.6837604e-03, 2.7722230e-03, 1.7858238e-03, 2.5218604e-03],
        [9.9807453e-01, 7.5011944e-06, 9.8636048e-04, 1.3320413e-04,
         6.5952525e-05, 1.0190145e-04, 1.5916865e-04, 1.9577592e-04,
         7.5997166e-05, 6.9961738e-05, 6.6107248e-05, 6.3516782e-05],
        [9.9968493e-01, 7.1249483e-06, 7.9754820e-05, 2.1535145e-05,
         1.7639193e-05, 3.3515182e-05, 2.8625092e-05, 3.2718515e-05,
         2.9747111e-05, 2.8389630e-05, 1.7621152e-05, 1.8502011e-05]]],
      dtype=float32)

In [181]:
model.predict_classes(np.array([x_val[0]]))

array([[2, 0, 0]])